In [2]:
import pymongo
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from pyquery import PyQuery as pq
from config import *
from urllib.parse import quote

# browser = webdriver.Chrome()
# browser = webdriver.PhantomJS(service_args=SERVICE_ARGS)

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
browser = webdriver.Chrome(chrome_options=chrome_options)

wait = WebDriverWait(browser, 10)
client = pymongo.MongoClient(MONGO_URL)
db = client[MONGO_DB]


def index_page(page):
    """
    抓取索引页
    :param page: 页码
    """
    print('正在爬取第', page, '页')
    try:
        url = 'https://s.taobao.com/search?q=' + quote(KEYWORD)
        browser.get(url)
        if page > 1:
            input = wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '#mainsrp-pager div.form > input')))
            submit = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '#mainsrp-pager div.form > span.btn.J_Submit')))
            input.clear()
            input.send_keys(page)
            submit.click()
        wait.until(
            EC.text_to_be_present_in_element((By.CSS_SELECTOR, '#mainsrp-pager li.item.active > span'), str(page)))
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.m-itemlist .items .item')))
        get_products()
    except TimeoutException:
        index_page(page)


def get_products():
    """
    提取商品数据
    """
    html = browser.page_source
    doc = pq(html)
    items = doc('#mainsrp-itemlist .items .item').items()
    for item in items:
        product = {
            'image': item.find('.pic .img').attr('data-src'),
            'price': item.find('.price').text(),
            'deal': item.find('.deal-cnt').text(),
            'title': item.find('.title').text(),
            'shop': item.find('.shop').text(),
            'location': item.find('.location').text()
        }
        print(product)
        save_to_mongo(product)


def save_to_mongo(result):
    """
    保存至MongoDB
    :param result: 结果
    """
    try:
        if db[MONGO_COLLECTION].insert(result):
            print('存储到MongoDB成功')
    except Exception:
        print('存储到MongoDB失败')


def main():
    """
    遍历每一页
    """
    for i in range(1, MAX_PAGE + 1):
        index_page(i)
    browser.close()


if __name__ == '__main__':
    main()

正在爬取第 1 页
{'image': '//g-search1.alicdn.com/img/bao/uploaded/i4/imgextra/i2/13022581/TB2lXJxuoOWBKNjSZKzXXXfWFXa_!!0-saturn_solar.jpg', 'price': '¥\n2988.00', 'deal': '5853人付款', 'title': '赠电动牙刷【2年保修】Apple/苹果 iPad 2018款 9.7英寸平板电脑 wifi平板电脑 吃鸡玩家', 'shop': '绿森数码官方旗舰店', 'location': '浙江 杭州'}


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:74: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


存储到MongoDB成功
{'image': '//g-search2.alicdn.com/img/bao/uploaded/i4/i4/1669409267/TB1G.9XuiQnBKNjSZFmXXcApVXa_!!0-item_pic.jpg', 'price': '¥\n2268.00', 'deal': '16467人付款', 'title': '[12期分期][两年保修]Apple/苹果\niPad\n2018款 9.7英寸wifi新款平板电脑32G/128G 正品国行新品授权旗舰店', 'shop': '卓辰数码旗舰店', 'location': '浙江 杭州'}
存储到MongoDB成功
{'image': '//g-search1.alicdn.com/img/bao/uploaded/i4/i4/1669409267/TB1_ZeJqbArBKNjSZFLXXc_dVXa_!!0-item_pic.jpg', 'price': '¥\n2738.00', 'deal': '1874人付款', 'title': '[12期分期]Apple/苹果\niPad\nmini 4 7.9英寸 平板电脑 128GWifi版', 'shop': '卓辰数码旗舰店', 'location': '浙江 杭州'}
存储到MongoDB成功
{'image': '//g-search2.alicdn.com/img/bao/uploaded/i4/i1/97045700/TB2qoUdyuuSBuNjSsplXXbe8pXa_!!97045700.jpg', 'price': '¥\n1868.00', 'deal': '3582人付款', 'title': 'Apple/苹果\niPad\n2018款 苹果平板电脑9.7英寸\nipad\n新款\nipad\n2018', 'shop': '深圳_恒波', 'location': '广东 深圳'}
存储到MongoDB成功
{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i3/1103997617/TB2b8U2slyWBuNkSmFPXXXguVXa_!!1103997617-0-item_pic.jpg', 'price': '¥\n49.00', '

{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i3/1963365327/TB2tv5pnQyWBuNjy0FpXXassXXa_!!1963365327.jpg', 'price': '¥\n3959.01', 'deal': '51人付款', 'title': 'Apple/苹果 新款\nipad\npro12.9寸pro 4G版 12.9pro 二代 平板电脑', 'shop': '宝诗丽1314', 'location': '广东 深圳'}
存储到MongoDB成功
{'image': '//g-search2.alicdn.com/img/bao/uploaded/i4/i2/277891388/TB2ooL.oUR1BeNjy0FmXXb0wVXa_!!277891388.jpg', 'price': '¥\n1678.00', 'deal': '250人付款', 'title': 'Apple/苹果\niPad\n2018款/平板电脑新款\niPad\n9.7 支持Pencil', 'shop': '我就爱购数码专营店', 'location': '江苏 南京'}
存储到MongoDB成功
{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i4/2539737278/TB22m_Dw90mpuFjSZPiXXbssVXa_!!2539737278.jpg', 'price': '¥\n4148.00', 'deal': '191人付款', 'title': '新款Apple/苹果\niPad\nPro 10.5寸Wifi 4G平板电脑 国行港版未激活', 'shop': '李小麦的店', 'location': '北京'}
存储到MongoDB成功
{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i3/3120180028/TB2Y7nKdgLD8KJjSszeXXaGRpXa_!!3120180028.jpg', 'price': '¥\n1650.00', 'deal': '234人付款', 'title': 'Apple/苹果\niPad\nAir 2 平板电脑9.7寸

{'image': '//g-search2.alicdn.com/img/bao/uploaded/i4/i3/1963365327/TB2tv5pnQyWBuNjy0FpXXassXXa_!!1963365327.jpg', 'price': '¥\n3959.01', 'deal': '51人付款', 'title': 'Apple/苹果 新款\nipad\npro12.9寸pro 4G版 12.9pro 二代 平板电脑', 'shop': '宝诗丽1314', 'location': '广东 深圳'}
存储到MongoDB成功
{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i2/277891388/TB2ooL.oUR1BeNjy0FmXXb0wVXa_!!277891388.jpg', 'price': '¥\n1678.00', 'deal': '250人付款', 'title': 'Apple/苹果\niPad\n2018款/平板电脑新款\niPad\n9.7 支持Pencil', 'shop': '我就爱购数码专营店', 'location': '江苏 南京'}
存储到MongoDB成功
{'image': '//g-search1.alicdn.com/img/bao/uploaded/i4/i4/2539737278/TB22m_Dw90mpuFjSZPiXXbssVXa_!!2539737278.jpg', 'price': '¥\n4148.00', 'deal': '191人付款', 'title': '新款Apple/苹果\niPad\nPro 10.5寸Wifi 4G平板电脑 国行港版未激活', 'shop': '李小麦的店', 'location': '北京'}
存储到MongoDB成功
{'image': '//g-search2.alicdn.com/img/bao/uploaded/i4/i3/3120180028/TB2Y7nKdgLD8KJjSszeXXaGRpXa_!!3120180028.jpg', 'price': '¥\n1650.00', 'deal': '234人付款', 'title': 'Apple/苹果\niPad\nAir 2 平板电脑9.7寸

...

{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i3/1963365327/TB2tv5pnQyWBuNjy0FpXXassXXa_!!1963365327.jpg', 'price': '¥\n3959.01', 'deal': '51人付款', 'title': 'Apple/苹果 新款\nipad\npro12.9寸pro 4G版 12.9pro 二代 平板电脑', 'shop': '宝诗丽1314', 'location': '广东 深圳'}
存储到MongoDB成功
{'image': '//g-search2.alicdn.com/img/bao/uploaded/i4/i2/277891388/TB2ooL.oUR1BeNjy0FmXXb0wVXa_!!277891388.jpg', 'price': '¥\n1678.00', 'deal': '250人付款', 'title': 'Apple/苹果\niPad\n2018款/平板电脑新款\niPad\n9.7 支持Pencil', 'shop': '我就爱购数码专营店', 'location': '江苏 南京'}
存储到MongoDB成功
{'image': '//g-search2.alicdn.com/img/bao/uploaded/i4/i4/2539737278/TB22m_Dw90mpuFjSZPiXXbssVXa_!!2539737278.jpg', 'price': '¥\n4148.00', 'deal': '191人付款', 'title': '新款Apple/苹果\niPad\nPro 10.5寸Wifi 4G平板电脑 国行港版未激活', 'shop': '李小麦的店', 'location': '北京'}
存储到MongoDB成功
{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i3/3120180028/TB2Y7nKdgLD8KJjSszeXXaGRpXa_!!3120180028.jpg', 'price': '¥\n1650.00', 'deal': '234人付款', 'title': 'Apple/苹果\niPad\nAir 2 平板电脑9.7寸

{'image': '//g-search1.alicdn.com/img/bao/uploaded/i4/i3/1963365327/TB2tv5pnQyWBuNjy0FpXXassXXa_!!1963365327.jpg', 'price': '¥\n3959.01', 'deal': '51人付款', 'title': 'Apple/苹果 新款\nipad\npro12.9寸pro 4G版 12.9pro 二代 平板电脑', 'shop': '宝诗丽1314', 'location': '广东 深圳'}
存储到MongoDB成功
{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i2/277891388/TB2ooL.oUR1BeNjy0FmXXb0wVXa_!!277891388.jpg', 'price': '¥\n1678.00', 'deal': '250人付款', 'title': 'Apple/苹果\niPad\n2018款/平板电脑新款\niPad\n9.7 支持Pencil', 'shop': '我就爱购数码专营店', 'location': '江苏 南京'}
存储到MongoDB成功
{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i4/2539737278/TB22m_Dw90mpuFjSZPiXXbssVXa_!!2539737278.jpg', 'price': '¥\n4148.00', 'deal': '191人付款', 'title': '新款Apple/苹果\niPad\nPro 10.5寸Wifi 4G平板电脑 国行港版未激活', 'shop': '李小麦的店', 'location': '北京'}
存储到MongoDB成功
{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i3/3120180028/TB2Y7nKdgLD8KJjSszeXXaGRpXa_!!3120180028.jpg', 'price': '¥\n1650.00', 'deal': '234人付款', 'title': 'Apple/苹果\niPad\nAir 2 平板电脑9.7寸

{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i3/1963365327/TB2tv5pnQyWBuNjy0FpXXassXXa_!!1963365327.jpg', 'price': '¥\n3959.01', 'deal': '51人付款', 'title': 'Apple/苹果 新款\nipad\npro12.9寸pro 4G版 12.9pro 二代 平板电脑', 'shop': '宝诗丽1314', 'location': '广东 深圳'}
存储到MongoDB成功
{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i2/277891388/TB2ooL.oUR1BeNjy0FmXXb0wVXa_!!277891388.jpg', 'price': '¥\n1678.00', 'deal': '250人付款', 'title': 'Apple/苹果\niPad\n2018款/平板电脑新款\niPad\n9.7 支持Pencil', 'shop': '我就爱购数码专营店', 'location': '江苏 南京'}
存储到MongoDB成功
{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i4/2539737278/TB22m_Dw90mpuFjSZPiXXbssVXa_!!2539737278.jpg', 'price': '¥\n4148.00', 'deal': '191人付款', 'title': '新款Apple/苹果\niPad\nPro 10.5寸Wifi 4G平板电脑 国行港版未激活', 'shop': '李小麦的店', 'location': '北京'}
存储到MongoDB成功
{'image': '//g-search1.alicdn.com/img/bao/uploaded/i4/i3/3120180028/TB2Y7nKdgLD8KJjSszeXXaGRpXa_!!3120180028.jpg', 'price': '¥\n1650.00', 'deal': '234人付款', 'title': 'Apple/苹果\niPad\nAir 2 平板电脑9.7寸

{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i3/1963365327/TB2tv5pnQyWBuNjy0FpXXassXXa_!!1963365327.jpg', 'price': '¥\n3959.01', 'deal': '51人付款', 'title': 'Apple/苹果 新款\nipad\npro12.9寸pro 4G版 12.9pro 二代 平板电脑', 'shop': '宝诗丽1314', 'location': '广东 深圳'}
存储到MongoDB成功
{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i2/277891388/TB2ooL.oUR1BeNjy0FmXXb0wVXa_!!277891388.jpg', 'price': '¥\n1678.00', 'deal': '250人付款', 'title': 'Apple/苹果\niPad\n2018款/平板电脑新款\niPad\n9.7 支持Pencil', 'shop': '我就爱购数码专营店', 'location': '江苏 南京'}
存储到MongoDB成功
{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i4/2539737278/TB22m_Dw90mpuFjSZPiXXbssVXa_!!2539737278.jpg', 'price': '¥\n4148.00', 'deal': '191人付款', 'title': '新款Apple/苹果\niPad\nPro 10.5寸Wifi 4G平板电脑 国行港版未激活', 'shop': '李小麦的店', 'location': '北京'}
存储到MongoDB成功
{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i3/3120180028/TB2Y7nKdgLD8KJjSszeXXaGRpXa_!!3120180028.jpg', 'price': '¥\n1650.00', 'deal': '234人付款', 'title': 'Apple/苹果\niPad\nAir 2 平板电脑9.7寸

{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i3/1963365327/TB2tv5pnQyWBuNjy0FpXXassXXa_!!1963365327.jpg', 'price': '¥\n3959.01', 'deal': '51人付款', 'title': 'Apple/苹果 新款\nipad\npro12.9寸pro 4G版 12.9pro 二代 平板电脑', 'shop': '宝诗丽1314', 'location': '广东 深圳'}
存储到MongoDB成功
{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i2/277891388/TB2ooL.oUR1BeNjy0FmXXb0wVXa_!!277891388.jpg', 'price': '¥\n1678.00', 'deal': '250人付款', 'title': 'Apple/苹果\niPad\n2018款/平板电脑新款\niPad\n9.7 支持Pencil', 'shop': '我就爱购数码专营店', 'location': '江苏 南京'}
存储到MongoDB成功
{'image': '//g-search2.alicdn.com/img/bao/uploaded/i4/i4/2539737278/TB22m_Dw90mpuFjSZPiXXbssVXa_!!2539737278.jpg', 'price': '¥\n4148.00', 'deal': '191人付款', 'title': '新款Apple/苹果\niPad\nPro 10.5寸Wifi 4G平板电脑 国行港版未激活', 'shop': '李小麦的店', 'location': '北京'}
存储到MongoDB成功
{'image': '//g-search2.alicdn.com/img/bao/uploaded/i4/i3/3120180028/TB2Y7nKdgLD8KJjSszeXXaGRpXa_!!3120180028.jpg', 'price': '¥\n1650.00', 'deal': '234人付款', 'title': 'Apple/苹果\niPad\nAir 2 平板电脑9.7寸